In [1]:
import numpy as np
import networkx as nx
from docplex.mp.model import Model
from collections import defaultdict


# Fully connected problems

In [2]:
def WMaxCut(G):
    # Weighted maxcut problem from networkx graph model as a QUBO problem
    mdl = Model('MaxCut')
    num_vertices = G.number_of_nodes()
    x = {i: mdl.binary_var(name=f"x_{i}") for i in range(num_vertices)}
    mdl.minimize(-mdl.sum(G[i][j]["weight"]*(x[i] * (1 - x[j]) + x[j] * (1 - x[i]))  for (i, j) in G.edges))
    return mdl

In [11]:
np.random.seed(1)
problems = defaultdict(dict)

nqs = range(10, 20)
for nq in nqs:
    print(f"Number of qubits: {nq}")
    G  = nx.Graph()
    G.add_nodes_from(range(nq))
    for i in range(nq):
        for j in range(i + 1, nq):
            w = np.random.choice([0.1, 0.2, 0.5, 2.0])
            G.add_weighted_edges_from([[i,j,w]])
    problems[nq]["G"] = G
    mdl = WMaxCut(G)
    mdl.solve(time_limit=60) # Solving using CPLEX
    problems[nq]["time_cplex"] = mdl.get_solve_details().deterministic_time
    problems[nq]["real_time_cplex"] = mdl.get_solve_details().time
    problems[nq]["n_iterations_cplex"] = mdl.get_solve_details().nb_iterations
    problems[nq]["status"] = mdl.get_solve_details().status
    problems[nq]["sol"] = "".join(str(round(mdl.solution.get_value(var))) for var in mdl.iter_binary_vars())
# np.save("./Data/WMC_FC.npy", problems)

Number of qubits: 10
Number of qubits: 11
Number of qubits: 12
Number of qubits: 13
Number of qubits: 14
Number of qubits: 15
Number of qubits: 16
Number of qubits: 17
Number of qubits: 18
Number of qubits: 19


# Native Layout Graph

In [5]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
service = QiskitRuntimeService()

In [15]:
backend_name = "ibm_fez"
backend = service.backend(backend_name)

In [17]:
np.random.seed(123)
n_qubits = backend.num_qubits
problem = {}
G = nx.Graph()
G.add_nodes_from(range(n_qubits))
for i,j in backend.coupling_map.get_edges(): 
    G.add_weighted_edges_from([[i, j, np.random.choice([0.1, 0.2, 0.3, 0.5, 1.0])]])

mdl = WMaxCut(G)
mdl.solve()
problem["G"] = G
problem["sol"] = "".join(str(round(mdl.solution.get_value(var))) for var in mdl.iter_binary_vars())
np.save(f"./Data/{backend_name}/NL-problem.npy", problem)